# **Customer Retention Toolkit Demo**

In [1]:
import pandas as pd
import os
import random
from customer_retention_toolkit.db.schema import *
from customer_retention_toolkit.db.sql_interactions import  SqlHandler
from customer_retention_toolkit.logger import *
from FillTables import *
from customer_retention_toolkit.models.MLWorkflow import MLWorkflow
from customer_retention_toolkit.api import app

# Configure the logger
logger = logging.getLogger('example_notebook')
logger.setLevel(logging.DEBUG)
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
ch.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))
logger.addHandler(ch)

## **Data Preparation and Ingestion**

In [2]:
db_filename = 'temp.db'

if os.path.exists(db_filename):
    os.remove(db_filename)
    logger.info(f"Deleted existing {db_filename}")

create_database()
InsertToTables()

2023-12-12 20:40:13,945 - INFO - Deleted existing temp.db


2023-12-12 20:40:13,983 - sql_interactions.py - get_table_columns - INFO - (The list of columns: ['StateID', 'StateName']) - line: 67
2023-12-12 20:40:13,986 - sql_interactions.py - get_table_columns - INFO - (The list of columns: ['StateID', 'StateName']) - line: 67
2023-12-12 20:40:13,995 - sql_interactions.py - insert_many - INFO - (Data inserted successfully) - line: 102
2023-12-12 20:40:13,997 - sql_interactions.py - close_cnxn - INFO - (Committing the changes) - line: 52
2023-12-12 20:40:13,999 - sql_interactions.py - close_cnxn - INFO - (The connection has been closed) - line: 55
2023-12-12 20:40:14,001 - sql_interactions.py - get_table_columns - INFO - (The list of columns: ['PlanID', 'AreaCode', 'InternationalPlan', 'VoiceMailPlan', 'NumberVMailMessages']) - line: 67
2023-12-12 20:40:14,004 - sql_interactions.py - get_table_columns - INFO - (The list of columns: ['PlanID', 'AreaCode', 'InternationalPlan', 'VoiceMailPlan', 'NumberVMailMessages']) - line: 67
2023-12-12 20:40:14,

## **Load data from the database**

In [3]:
dbname = 'temp'
workflow = MLWorkflow(dbname)

## **Run the workflow**

In [4]:
table_names = ['State', 'PlanDetails', 'DayUsage', 'EveUsage', 'NightUsage', 'IntlUsage', 'CustomerMetrics']
metrics, X_test, y_test, best_model_predictions = workflow.run_workflow(table_names)

logger.info("Random Forest Model Metrics:")
for metric_name, metric_value in metrics.items():
    logger.info(f"{metric_name}: {metric_value}")

2023-12-12 20:40:18,868 - MLWorkflow.py - INFO - Model trained successfully.
2023-12-12 20:40:18,917 - INFO - Random Forest Model Metrics:
2023-12-12 20:40:18,919 - INFO - Accuracy: 0.9138576779026217
2023-12-12 20:40:18,920 - INFO - Confusion Matrix: [[454   1]
 [ 45  34]]
2023-12-12 20:40:18,920 - INFO - Classification Report:               precision    recall  f1-score   support

         0.0       0.91      1.00      0.95       455
         1.0       0.97      0.43      0.60        79

    accuracy                           0.91       534
   macro avg       0.94      0.71      0.77       534
weighted avg       0.92      0.91      0.90       534



## **Save predictions to the database**

In [5]:
workflow.save_predictions_to_db(X_test, y_test, best_model_predictions, table_name='PredictionResults')    

2023-12-12 20:40:20,657 - sql_interactions.py - get_table_columns - INFO - (The list of columns: ['PredictionID', 'CustomerID', 'PredictedLabel', 'ModelName', 'ChurnStatus']) - line: 67
2023-12-12 20:40:20,661 - sql_interactions.py - insert_many - INFO - (Data inserted successfully) - line: 102
2023-12-12 20:40:20,662 - sql_interactions.py - close_cnxn - INFO - (Committing the changes) - line: 52
2023-12-12 20:40:20,665 - sql_interactions.py - close_cnxn - INFO - (The connection has been closed) - line: 55
2023-12-12 20:40:20,667 - MLWorkflow.py - INFO - Predictions saved to PredictionResults


## **Example API Test**

In [7]:
import requests

# The base URL for your API
base_url = "http://127.0.0.1:5000"

# GET request to the root endpoint
response = requests.get(f"{base_url}/")
print(response.json())


{'message': 'Initializing'}


In [8]:
# Replace with a valid customer ID
customer_id = 1
response = requests.get(f"{base_url}/get_data/{customer_id}")
print(response.json())


{'CustomerID': 1, 'StateID': 1, 'PlanID': 1, 'DayUsageID': 1, 'EveUsageID': 1, 'NightUsageID': 1, 'IntlUsageID': 1, 'CustomerServiceCalls': 1, 'ChurnStatus': 0}


In [9]:
# base_url = 'http://127.0.0.1:5000'  # Replace with the actual base URL if different

new_customer_data = {
    "CustomerID": 2749,
    "ChurnStatus": 1,
    "StateID": 1,  # Assuming '1' is a valid StateID in your database
    "PlanID": 1,  # Assuming '1' is a valid PlanID in your database
    "DayUsageID": 1,  # Assuming '1' is a valid DayUsageID in your database
    "EveUsageID": 1,  # Assuming '1' is a valid EveUsageID in your database
    "NightUsageID": 1,  # Assuming '1' is a valid NightUsageID in your database
    "IntlUsageID": 1,  # Assuming '1' is a valid IntlUsageID in your database
    "CustomerServiceCalls": 1  # Number of customer service calls
}

response = requests.post(f"{base_url}/create_data", json=new_customer_data)
print(response.json())


{'message': 'Record created successfully'}


In [10]:
# PUT request to update a customer record
update_data = {
    "column_name": "ChurnStatus",
    "new_value": 1,
    "CustomerID": 2749  # Make sure this ID exists in your database
}
response = requests.put(f"{base_url}/update_data", json=update_data)
print(response.json())


{'message': 'Record updated successfully'}


In [11]:
# GET request to predict churn for a customer
# response = requests.get(f"{base_url}/predict_churn/{customer_id}")
# print(response.json())
customer_id = 555
response = requests.get(f"{base_url}/predict_churn/{customer_id}")
print('Status Code:', response.status_code)

if response.status_code == 200:
    try:
        data = response.json()
        print(data)
    except JSONDecodeError:
        print('Response could not be decoded as JSON:', response.text)
else:
    print('Failed to fetch data:', response.text)


Status Code: 200
{'CustomerID': 555, 'ChurnPrediction': 0}
